In [266]:
# !pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13486 sha256=4e607befcc1cdc819f48670839a5d29378401945b7f936daa078ab46c89f71f6
  Stored in directory: c:\users\123gu\appdata\local\pip\cache\wheels\a0\b8\b7\8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent


You should consider upgrading via the 'c:\users\123gu\anaconda3\python.exe -m pip install --upgrade pip' command.


In [267]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
from datetime import datetime

import pandas as pd
import numpy as np

### 크롤링1
1은 전부
2는 성명, 발행회사와의 관계, 업무상 연락처 및 담당자
3은 다.세부변동내역 전체

In [272]:
# 크롬 드라이버로 웹 브라우저 실행

# 크롤링 ip차단 우회 시도
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')

path = './lib/chromedriver.exe'
driver = webdriver.Chrome(chrome_options=options, executable_path = path)
# driver = webdriver.Chrome(path) # 원래 코드

df_company = pd.DataFrame(columns = ['발행회사', '법인구분', '회사코드', '발행주식 총수'])
df_reporter = pd.DataFrame(columns = ['발행회사', '성명(한글)', '성명(한자,영문)', '생년월일 또는 사업자등록번호 등', '임원(등기여부)', '직위명', '선임일', '퇴임일', '주요주주', '소속회사', '부서', '직위', '전화번호', '성명', '팩스번호', '이메일 주소'])     
df_rows = pd.DataFrame(columns = ['발행회사', '보고사유', '변동일', '특정증권등의 종류', '소유주식수(주) 변동전', '소유주식수(주) 증감', '소유주식수(주) 변동후', '취득/처분단가(원)', '비고'])    
df_sum = pd.DataFrame(columns = ['발행회사', '소유주식수(주) 변동전', '소유주식수(주) 증감', '소유주식수(주) 변동후', '취득/처분단가(원)', '비고'])

page = 1
while page <= 3:
    #driver = webdriver.Chrome('chromedriver', chrome_options=options)
    url= "https://dart.fss.or.kr/dsab007/main.do?option=corp"
    driver.get(url)
    time.sleep(1)

    # 공시통합검색란에서 지분공시 클릭 > 임원·주요주주특정증권등소유상황보고서 클릭 > 기간설정 > 검색 버튼 누르기
    driver.find_element_by_xpath('//*[@id="li_04"]/label').click()
    driver.find_element_by_xpath('//*[@id="divPublicTypeDetail_04"]/ul/li[4]/span/label').click()

    # 기간설정
    element = driver.find_element_by_xpath('//*[@id="startDate"]') # start day text input 창 선택
    length = len(element.get_attribute('value'))# start day text input 창 선택 초기화
    element.send_keys(length * Keys.BACKSPACE)
    element.send_keys("20210101")# start day text input 값 insert

    element = driver.find_element_by_xpath('//*[@id="endDate"]') # end day text input 창 선택
    length = len(element.get_attribute('value')) # end day text input 창 선택 초기화
    element.send_keys(length * Keys.BACKSPACE)
    element.send_keys("20211028") # end day text input 값 insert

    # 검색창 클릭
    driver.find_element_by_xpath('//*[@id="searchForm"]/div[1]/div/ul/li/div[3]/a').click()
    time.sleep(2)

    # 
    #driver.find_element_by_xpath('//*[@id="r_20211018000266"]').click()

    # 현재 페이지에 있는 소스를 모두 긁어온다
    source = driver.page_source
    html_s = BeautifulSoup(source, 'lxml')

    # select로 지정 : td.tL의 자식 중 a인거 모두 가져와라
    title_list = html_s.select('td.tL > a')

    # href만 추출
    hre = []
    for i  in range(0,len(title_list)) :
        hre.append(title_list[i].get('href'))
    hre # 15개

    # 추출한 href를 주소에 붙여 크롤링할 내용이 있는 페이지 열기
    new_url_base = 'https://dart.fss.or.kr'
    for i in range(0, len(hre)):
        new_url = new_url_base + hre[i]
        driver.get(new_url)
        time.sleep(2)
        
        # 문서목차 버튼 클릭
        driver.find_element_by_id('wideBtn').click()
        time.sleep(2)

        ### 1. 발행회사에 관한 사항 클릭 ###
        driver.find_element_by_id('2_anchor').click()
        time.sleep(2)

        # iframe 안으로 진입
        iframes = driver.find_elements_by_tag_name('iframe')[0]
        driver.switch_to.frame(iframes)

        # 발행회사명
        one_1 = driver.find_element_by_xpath('/html/body/table/tbody/tr[1]/td[2]').text

        # 법인구분, 회사코드
        one_2_1 = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[2]').text
        one_2_2 = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[4]').text

        # 발행주식 총수
        one_3 = driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td[2]').text

        # dataframe 생성
        df_company_new = pd.DataFrame({'발행회사' : one_1, '법인구분' : one_2_1, '회사코드' : one_2_2, '발행주식 총수' : one_3}, index=[0])
        df_company = pd.concat([df_company, df_company_new])

        # iframe 나옴
        driver.switch_to.default_content()

        # 문서목차버튼 클릭
        driver.find_element_by_id('wideBtn').click()
        time.sleep(2)

        ### 2. 보고자에 관한 사항 클릭 ###
        driver.find_element_by_id('3_anchor').click()
        time.sleep(2)

        # iframe 안으로 진입
        iframes = driver.find_elements_by_tag_name('iframe')[0]
        driver.switch_to.frame(iframes)

        ## 성명(명칭)
        # 한글
        two_1_1 = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[3]').text
        # 한자(영문)
        two_1_2 = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[5]').text
        # 생년월일 또는 사업자등록번호 등
        two_1_3 = driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td[2]').text

        ## 발행회사와의 관계
        # 임원(등기여부)
        two_2_1 = driver.find_element_by_xpath('/html/body/table/tbody/tr[5]/td[3]').text
        # 직위명
        two_2_2 = driver.find_element_by_xpath('/html/body/table/tbody/tr[5]/td[5]').text
        # 선임일
        two_2_3 = driver.find_element_by_xpath('/html/body/table/tbody/tr[6]/td[2]').text
        # 퇴임일
        two_2_4 = driver.find_element_by_xpath('/html/body/table/tbody/tr[6]/td[4]').text
        # 주요주주
        two_2_5 = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td[2]').text

        ## 업무상 연락처 및 담당자
        # 소속회사
        two_3_1 = driver.find_element_by_xpath('/html/body/table/tbody/tr[8]/td[3]').text
        # 부서
        two_3_2 = driver.find_element_by_xpath('/html/body/table/tbody/tr[8]/td[5]').text
        # 직위
        two_3_3 = driver.find_element_by_xpath('/html/body/table/tbody/tr[9]/td[2]').text
        # 전화번호
        two_3_4 = driver.find_element_by_xpath('/html/body/table/tbody/tr[9]/td[4]').text
        # 성명
        two_3_5 = driver.find_element_by_xpath('/html/body/table/tbody/tr[10]/td[2]').text
        # 팩스번호
        two_3_6 = driver.find_element_by_xpath('/html/body/table/tbody/tr[10]/td[4]').text
        # 이메일 주소
        two_3_7 = driver.find_element_by_xpath('/html/body/table/tbody/tr[11]/td[2]').text

        # dataframe 하나로 정리
        df_reporter_new = pd.DataFrame({'발행회사' : one_1, '성명(한글)' : two_1_1, '성명(한자,영문)' : two_1_2, '생년월일 또는 사업자등록번호 등' : two_1_3, '임원(등기여부)' : two_2_1, '직위명' : two_2_2, '선임일' : two_2_3, '퇴임일' : two_2_4, '주요주주' : two_2_5, '소속회사' : two_3_1, '부서' : two_3_2, '직위' : two_3_3, '전화번호' : two_3_4, '성명' : two_3_5, '팩스번호' : two_3_6, '이메일 주소' : two_3_7}, index=[0])
        df_reporter = pd.concat([df_reporter, df_reporter_new])

        # iframe 나옴
        driver.switch_to.default_content()

        # 문서목차버튼 클릭
        driver.find_element_by_id('wideBtn').click()
        time.sleep(2)

        ### 3. 특정증권등의 소유상황 클릭 ###
        driver.find_element_by_id('4_anchor').click()
        time.sleep(2)

        # iframe 안으로 진입
        iframes = driver.find_elements_by_tag_name('iframe')[0]
        driver.switch_to.frame(iframes)

        ## 다. 세부변동내역
        table = driver.find_element_by_xpath('/html/body/table[4]')
        tbody = table.find_element_by_tag_name("tbody")
        rows = tbody.find_elements_by_tag_name("tr")

        three_1_1 = [] 
        three_1_2 = []
        three_1_3 = []
        three_1_4 = []
        three_1_5 = []
        three_1_6 = []
        three_1_7 = []
        three_1_8 = []
        three_2_1 = []
        three_2_2 = []
        three_2_3 = []
        three_2_4 = []
        three_2_5 = []

        name = [one_1]
        names = name * (len(rows)-1)

        for index, value in enumerate(rows):
            if index < len(rows)-1 : 
                # 보고사유
                three_1_1.append(value.find_elements_by_tag_name("td")[0].text)
                # 변동일
                three_1_2.append(value.find_elements_by_tag_name("td")[1].text)
                # 특정증권등의 종류
                three_1_3.append(value.find_elements_by_tag_name("td")[2].text)
                # 소유주식수(주) 변동전
                three_1_4.append(value.find_elements_by_tag_name("td")[3].text)
                # 소유주식수(주) 증감
                three_1_5.append(value.find_elements_by_tag_name("td")[4].text)
                # 소유주식수(주) 변동후
                three_1_6.append(value.find_elements_by_tag_name("td")[5].text)
                # 취득/처분단가(원)
                three_1_7.append(value.find_elements_by_tag_name("td")[6].text)
                # 비고
                three_1_8.append(value.find_elements_by_tag_name("td")[7].text)
            if index == len(rows)-1 :
                ## 합계
                # 소유주식수(주) 변동전
                three_2_1.append(value.find_elements_by_tag_name("td")[1].text)
                # 소유주식수(주) 증감
                three_2_2.append(value.find_elements_by_tag_name("td")[2].text)
                # 소유주식수(주) 변동후
                three_2_3.append(value.find_elements_by_tag_name("td")[3].text)
                # 취득/처분단가(원)
                three_2_4.append(value.find_elements_by_tag_name("td")[4].text)
                # 비고
                three_2_5.append(value.find_elements_by_tag_name("td")[5].text)

        # 원데이터 row들만 있는 dataframe
        df_rows_new = pd.DataFrame({'발행회사' : names, '보고사유' : three_1_1, '변동일' : three_1_2, '특정증권등의 종류' : three_1_3, '소유주식수(주) 변동전' : three_1_4, '소유주식수(주) 증감' : three_1_5, '소유주식수(주) 변동후' : three_1_6, '취득/처분단가(원)' : three_1_7, '비고' : three_1_8})
        df_rows = pd.concat([df_rows, df_rows_new])
        # 합계만 따로 저장한 dataframe
        df_sum_new = pd.DataFrame({'발행회사' : one_1, '소유주식수(주) 변동전' : three_2_1, '소유주식수(주) 증감' : three_2_2, '소유주식수(주) 변동후' : three_2_3, '취득/처분단가(원)' : three_2_4, '비고' : three_2_5}, index=[0])
        df_sum = pd.concat([df_sum, df_sum_new])
        # iframe 나옴
        driver.switch_to.default_content()

        print(page, i)
   
    page += 1
    
driver.quit()

<ipython-input-272-655570319fd3>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path = path)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="li_04"]/label"}
  (Session info: chrome=94.0.4606.81)


In [ ]:
df_company

In [273]:
df_reporter

,발행회사,성명(한글),"성명(한자,영문)",생년월일 또는 사업자등록번호 등,임원(등기여부),직위명,선임일,퇴임일,주요주주,소속회사,부서,직위,전화번호,성명,팩스번호,이메일 주소


In [ ]:
df_rows

In [ ]:
df_sum

In [274]:
# openAPI 짜다만거
# 오픈다트 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS004&apiId=2019022

import requests
import xmltodict
from datetime import datetime, timedelta
import os
import json
import sys

In [ ]:
base_url = 'https://opendart.fss.or.kr/api/elestock.json?crtfc_key='
certf = '9c3fec15baa8e7556c233b07b29fb96fde4de8d5'
corp = '&corp_code=00126380'

url = base_url + certf + corp
response = requests.get(url)

# ensure_ascii = False : 파일이 저장될때 한글이 유니코드 형태로 저장되지 않도록 함
if response.status_code == 200:
    xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
    xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
    xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환

    file_path = "./data/일별농업기상/weather{}_{}.json".format(spot[j],start)
    with open(file_path, 'w', encoding='utf-8') as outfile:
        json.dump(xdjd, outfile, indent='\t', ensure_ascii=False)
    print(start,spot[j])
else:
    print(response.status_code)